### Clarification Regarding Entropy

The [video](https://youtu.be/1s2dRczcu1A) mentions that entropy will decrease after the motion update step and that entropy will increase after measurement step. What is meant is that that entropy will decrease after the measurement update (sense) step and that entropy will increase after the movement step (move).

In general, entropy represents the amount of uncertainty in a system. Since the measurement update step decreases uncertainty, entropy will decrease. The movement step increases uncertainty, so entropy will increase after this step.

Let's look at our current example where the robot could be at one of five different positions. The maximum uncertainty occurs when all positions have equal probabilities 

Following the formula $Entropy = -\sum p(x_i)logp(x_i)$ we get: -5 * 0.2 * log (0.2) = 0.699

Taking a measurement will decrease uncertainty and entropy. Let's say after taking a measurement, the probabilities become "[0.05, 0.05, 0.05, 0.8, 0.05]". Now we have a more certain guess as to where the robot is located and our entropy has decreased to 0.338.

This is the theory in BAYESIAN THINKING video 35 but I cannot understand how this connects to the explanatory [video](https://youtu.be/K8g3Hss8Q1A) and the results below

In [1]:
# set all variables as global variables
p=[0.2, 0.2, 0.2, 0.2, 0.2]
world=['green', 'red', 'red', 'green', 'green']
measurements = ['red', 'green']
motions = [1, 1]
pHit = 0.6
pMiss = 0.2
pExact = 0.8
pOvershoot = 0.1
pUndershoot = 0.1

In [2]:
# your solution
def sense_m(p, world, Z):
    q = list()
    for i in range(len(p)):
        hit = (Z == world[i])
        q.append(p[i] * (hit * pHit + (1-hit) * pMiss))
    
    total = sum(q)
    for i, v in enumerate(q):
        q[i] = q[i]/total
    return q


def move_m(p, U):
    q = []
    for i in range(len(p)):
        s = pExact * p[(i-U) % len(p)]
        s = s + pOvershoot * p[(i-U-1) % len(p)]
        s = s + pUndershoot * p[(i-U+1) % len(p)]
        q.append(s)
    return q

In [3]:
# my solution
def sense_c(prior, world, color):
    multiplier = lambda color: pHit if color == 'red' else pMiss # create multiplier function
    multiplier_list = list(map(multiplier, world)) # calculate multiplier list
    priorPerMultiplier = list(map(lambda x,y:x*y,prior,multiplier_list)) # update after new multiplier (evidence)
    total = sum(priorPerMultiplier) # calculate sum of normalized values
    posterior = list(map(lambda i:i/total,priorPerMultiplier))
    return posterior

def move_c(p, U):
    sE = [pExact * p[(i - U) % len(p)] for i in range(len(p))]
    sU = [pUndershoot * p[(i - U + 1) % len(p)] for i in range(len(p))]
    sO = [pOvershoot * p[(i - U - 1) % len(p)] for i in range(len(p))]
    s = list(map(lambda x, y, z: x + y + z, sE, sU, sO ))
    
    return s

In [4]:
# Is my sense exactly your sense?
sense_c(p, world, measurements[0]) == sense_m(p, world, measurements[0])

True

In [5]:
sense_c is sense_m

False

In [6]:
# Is my move exactry your move?
move_c(p,motions[0]) == move_m(p,motions[0])

True

In [7]:
move_c is move_m

False

In [8]:
p_m = [0.2, 0.2, 0.2, 0.2, 0.2]
for k in range(len(measurements)):
    p_m = sense_m(p_m, world, measurements[k])
    print(p_m)
    p_m = move_m(p_m, motions[k])
    print(p_m)
print(p_m)

[0.1111111111111111, 0.3333333333333332, 0.3333333333333332, 0.1111111111111111, 0.1111111111111111]
[0.11111111111111112, 0.13333333333333333, 0.311111111111111, 0.311111111111111, 0.1333333333333333]
[0.1578947368421053, 0.06315789473684212, 0.1473684210526316, 0.44210526315789467, 0.1894736842105263]
[0.21157894736842103, 0.1515789473684211, 0.08105263157894739, 0.16842105263157897, 0.3873684210526316]
[0.21157894736842103, 0.1515789473684211, 0.08105263157894739, 0.16842105263157897, 0.3873684210526316]


In [9]:
p_c = [0.2, 0.2, 0.2, 0.2, 0.2]
for k in range(len(measurements)):
    p_c = sense_c(p_c, world, measurements[k])
    print(p_c)
    p_c = move_c(p_c, motions[k])
    print(p_c)
print(p_c)

[0.1111111111111111, 0.3333333333333332, 0.3333333333333332, 0.1111111111111111, 0.1111111111111111]
[0.11111111111111112, 0.1333333333333333, 0.311111111111111, 0.311111111111111, 0.13333333333333333]
[0.058823529411764726, 0.2117647058823529, 0.49411764705882344, 0.16470588235294117, 0.07058823529411766]
[0.07882352941176472, 0.07529411764705884, 0.22470588235294117, 0.4329411764705882, 0.18823529411764706]
[0.07882352941176472, 0.07529411764705884, 0.22470588235294117, 0.4329411764705882, 0.18823529411764706]


In [10]:
p_m is p_c

False

In [11]:
p_m == p_c

False

My dear mentor it looks that while senses are equal (not identical) we move quite differently!